# Simple classification with CNN

 An example of classification using a convolutional neural network for the famous MNIST dataset

## Objectives :

- Recognizing handwritten numbers
- Understanding the principle of a classifier CNN network 
- Implementation with Keras

The MNIST dataset (Modified National Institute of Standards and Technology) is a must for Deep Learning.It consists of 60,000 small images of handwritten numbers for learning and 10,000 for testing.

# Step 1 - Init python stuff

In [3]:
import tensorflow as tf
from tensorflow import keras

import numpy as np
import matplotlib.pyplot as plt
import sys,os
from importlib import reload

Verbosity during training: 0 = silent, 1 = progress bar, 2 = one line per epoch

In [4]:
fit_verbosity = 1

# Step 2 - Retrieve data

MNIST is one of the most famous historic dataset. Include in Keras datasets.

In [ ]:
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

x_train = x_train /zs